<a href="https://colab.research.google.com/github/hamednasr/TensorFlow-Projects/blob/main/MobileNetV2_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MobileNetv2 feature extraction transfer learning

### **mission** :we want to classify the images of ten different foods, using the MobileNetv2 and transfer learning, using only 10% of the training data which is only 75 photos in each class, the test set contains 250 images in each class. 

In [6]:
import tensorflow as tf
import tensorflow_hub as hub
import zipfile
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import datetime
from tensorflow.keras import layers, optimizers

##load data:

In [2]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

--2022-02-28 06:17:58--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.153.128, 142.250.145.128, 173.194.69.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.153.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M  68.8MB/s    in 2.3s    

2022-02-28 06:18:00 (68.8 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [3]:
zip_ref = zipfile.ZipFile('10_food_classes_10_percent.zip')
zip_ref.extractall()
zip_ref.close()

In [4]:
for dirpath, dirnames,filenames in os.walk('10_food_classes_10_percent'):
  print(f'there are {len(dirnames)} folders and {len(filenames)} images in {dirpath}')

there are 2 folders and 0 images in 10_food_classes_10_percent
there are 10 folders and 0 images in 10_food_classes_10_percent/train
there are 0 folders and 75 images in 10_food_classes_10_percent/train/ice_cream
there are 0 folders and 75 images in 10_food_classes_10_percent/train/fried_rice
there are 0 folders and 75 images in 10_food_classes_10_percent/train/grilled_salmon
there are 0 folders and 75 images in 10_food_classes_10_percent/train/hamburger
there are 0 folders and 75 images in 10_food_classes_10_percent/train/chicken_curry
there are 0 folders and 75 images in 10_food_classes_10_percent/train/pizza
there are 0 folders and 75 images in 10_food_classes_10_percent/train/sushi
there are 0 folders and 75 images in 10_food_classes_10_percent/train/ramen
there are 0 folders and 75 images in 10_food_classes_10_percent/train/steak
there are 0 folders and 75 images in 10_food_classes_10_percent/train/chicken_wings
there are 10 folders and 0 images in 10_food_classes_10_percent/test


## Preprocess data:

In [7]:
train_datagen = ImageDataGenerator(rescale=1/255)
test_datagen = ImageDataGenerator(rescale=1/255)

In [8]:
train_dir = '10_food_classes_10_percent/train'
test_dir = '10_food_classes_10_percent/test'

In [9]:
train_data = train_datagen.flow_from_directory(train_dir,target_size=(224,224))
test_data = test_datagen.flow_from_directory(test_dir,target_size=(224,224))

Found 750 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.


In [ ]:
https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/5